In [1]:
import pandas as pd
import ast

In [2]:
# Lista para contener los diccionarios JSON de cada línea
data_list = []

# Ruta del archivo JSON
file_path = 'australian_users_items.json'

# Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

# Generar un DataFrame a partir de una lista de diccionarios
data_it = pd.DataFrame(data_list)
# Los datos de la columna ['ítems'] se desanidan.
data_it1 = data_it.explode(['items'])
data_it2 = pd.json_normalize(data_it1['items']).set_index(data_it1['items'].index)
data_it3= pd.concat([data_it2, data_it1], axis=1)

# Se excluyen las columnas que no son relevantes para el análisis posterior
data_it3.drop(['items_count','user_url','item_name','steam_id'], axis=1, inplace=True)

# Se eliminan los valores nulos por columna
data_it3 = data_it3.dropna(subset=['item_id'])
data_it3 = data_it3.dropna(subset=['playtime_forever'])

# Se excluyen dos columnas adicionales del DataFrame
data_it3.drop(['items','playtime_2weeks'], axis=1, inplace=True)

# Se establece el DataFrame solo para los juegos en los que se registró un tiempo jugado, aplicando una condición
data_it3 = data_it3[data_it3['playtime_forever'] != 0]

# Se retienen las tres columnas más relevantes para el análisis, descartando las demás
data_it3 = data_it3[['user_id','item_id','playtime_forever']]

# Los duplicados y los valores nulos, en caso de existir, se eliminan del DataFrame
data_it3 = data_it3.drop_duplicates()
data_it3 = data_it3.dropna()

# se exporta la tabla como archivo separa por comas de nombre 'user_items'
data_it3.to_csv('user_items.csv', index=False)